In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [2]:
# Load and preprocess the dataset
fulldata = pd.read_csv("KRS Dam - Time Series - Transposed.csv")
fulldata

,PARAMETERS,LOCATIONS,STATE,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,1,"KRS DAM, BALAMURIKSHETRA",KARNATAKA,0.17,0.19,0.12,0.15,0.15,0.06,0.1,0.1,0.1,0.09,0.50,0.32
1,2,"KRS DAM, BALAMURIKSHETRA",KARNATAKA,1.00,1.00,1.00,2.00,2.00,2.00,3.0,2.6,2.2,1.80,2.60,1.50
2,3,"KRS DAM, BALAMURIKSHETRA",KARNATAKA,8.90,8.90,8.90,8.70,8.90,8.90,8.7,8.5,8.5,8.50,8.40,8.30
3,4,"KRS DAM, BALAMURIKSHETRA",KARNATAKA,30.00,30.00,28.00,28.00,29.00,30.00,30.0,28.0,30.0,27.00,28.00,28.00
4,5,"KRS DAM, BALAMURIKSHETRA",KARNATAKA,6.80,7.90,8.10,7.50,7.10,7.40,7.3,8.0,6.5,7.40,7.70,7.70
5,6,"KRS DAM, BALAMURIKSHETRA",KARNATAKA,7.00,1600.00,1600.00,1100.00,470.00,840.00,1100.0,630.0,270.0,700.00,940.00,790.00
6,7,"KRS DAM, BALAMURIKSHETRA",KARNATAKA,26.00,25.00,24.00,25.00,24.00,26.00,23.0,23.0,18.0,24.00,22.00,24.00
7,8,"KRS DAM, BALAMURIKSHETRA",KARNATAKA,1600.00,1600.00,1600.00,1600.00,2200.00,1500.00,2400.0,1700.0,340.0,1200.00,1500.00,1300.00
8,9,"KRS DAM, BALAMURIKSHETRA",KARNATAKA,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.3,1.0,1.00,1.20,1.20
9,10,"KRS DAM, BALAMURIKSHETRA",KARNATAKA,319.00,380.00,322.00,285.00,359.00,300.00,396.0,312.0,84.0,298.00,327.00,295.00


In [3]:
for i in range(16):    
    print("--------------------------PARAMTER " + str(i)+"----------------------------------------------")
    years = fulldata.iloc[i][3:].index.to_numpy()

    years = [int(i) for i in years]

    dict_data = {'Date':years, 'Parameter':fulldata.iloc[i][3:]}

    df = pd.DataFrame(dict_data)
    df.to_csv("RNN_Dataset/Parameter_"+ str(i)+".csv",index=False)

    from sklearn.preprocessing import MinMaxScaler

    # Load the dataset
    data = pd.read_csv("RNN_Dataset/Parameter_"+ str(i)+".csv", index_col='Date', parse_dates=['Date'])

    # Prepare the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data['Parameter'].values.reshape(-1, 1))

    # Create the training and testing datasets
    train_size = int(len(scaled_data))
    train_data = scaled_data[:train_size, :]
    test_data = scaled_data[train_size:, :]


    train_data = train_data.reshape(-1, 1)
    test_data = test_data.reshape(-1, 1)

    def create_dataset(dataset, time_steps=1):
        X, y = [], []
        for i in range(len(dataset)-time_steps):
            a = dataset[i:(i+time_steps), 0]
            X.append(a)
            y.append(dataset[i+time_steps, 0])
        return np.array(X), np.array(y)

    time_steps = 2
    X_train, y_train = create_dataset(train_data, time_steps)
    X_test, y_test = create_dataset(test_data, time_steps)

    # Reshape the data
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    #X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=2, shuffle=False)

    train_predict = model.predict(X_train)
    #test_predict = model.predict(X_test)

    train_predict = scaler.inverse_transform(train_predict)
    y_train = scaler.inverse_transform([y_train])
    #test_predict = scaler.inverse_transform(test_predict)
    #y_test = scaler.inverse_transform([y_test])

    import matplotlib.pyplot as plt
    plt.plot(data.index[:train_size], data['Parameter'][:train_size], label='Training Data')
    #plt.plot(data.index[train_size+time_steps:], data['Parameter'][train_size+time_steps:], label='Testing Data')
    plt.plot(data.index[time_steps:train_size], train_predict[:,0], label='Training Predictions')
    #plt.plot(data.index[train_size+time_steps:], test_predict[:,0], label='Testing Predictions')
    plt.title("Parameter "+str(i))
    plt.legend()
    #plt.show()
    plt.savefig("RNN_outputs/"+"Parameter "+str(i)+".png")
    plt.close()
    print("-----------------------------------------------------------------------------------")

--------------------------PARAMTER 0----------------------------------------------
Epoch 1/100
1/1 - 4s - loss: 0.1451 - 4s/epoch - 4s/step
Epoch 2/100
1/1 - 0s - loss: 0.1418 - 26ms/epoch - 26ms/step
Epoch 3/100
1/1 - 0s - loss: 0.1386 - 25ms/epoch - 25ms/step
Epoch 4/100
1/1 - 0s - loss: 0.1353 - 30ms/epoch - 30ms/step
Epoch 5/100
1/1 - 0s - loss: 0.1321 - 28ms/epoch - 28ms/step
Epoch 6/100
1/1 - 0s - loss: 0.1289 - 32ms/epoch - 32ms/step
Epoch 7/100
1/1 - 0s - loss: 0.1258 - 29ms/epoch - 29ms/step
Epoch 8/100
1/1 - 0s - loss: 0.1226 - 30ms/epoch - 30ms/step
Epoch 9/100
1/1 - 0s - loss: 0.1195 - 29ms/epoch - 29ms/step
Epoch 10/100
1/1 - 0s - loss: 0.1164 - 28ms/epoch - 28ms/step
Epoch 11/100
1/1 - 0s - loss: 0.1133 - 26ms/epoch - 26ms/step
Epoch 12/100
1/1 - 0s - loss: 0.1103 - 27ms/epoch - 27ms/step
Epoch 13/100
1/1 - 0s - loss: 0.1073 - 27ms/epoch - 27ms/step
Epoch 14/100
1/1 - 0s - loss: 0.1045 - 26ms/epoch - 26ms/step
Epoch 15/100
1/1 - 0s - loss: 0.1017 - 28ms/epoch - 28ms/step


Epoch 29/100
1/1 - 0s - loss: 0.0910 - 31ms/epoch - 31ms/step
Epoch 30/100
1/1 - 0s - loss: 0.0836 - 26ms/epoch - 26ms/step
Epoch 31/100
1/1 - 0s - loss: 0.0772 - 27ms/epoch - 27ms/step
Epoch 32/100
1/1 - 0s - loss: 0.0719 - 29ms/epoch - 29ms/step
Epoch 33/100
1/1 - 0s - loss: 0.0678 - 24ms/epoch - 24ms/step
Epoch 34/100
1/1 - 0s - loss: 0.0651 - 40ms/epoch - 40ms/step
Epoch 35/100
1/1 - 0s - loss: 0.0636 - 26ms/epoch - 26ms/step
Epoch 36/100
1/1 - 0s - loss: 0.0635 - 25ms/epoch - 25ms/step
Epoch 37/100
1/1 - 0s - loss: 0.0644 - 35ms/epoch - 35ms/step
Epoch 38/100
1/1 - 0s - loss: 0.0660 - 29ms/epoch - 29ms/step
Epoch 39/100
1/1 - 0s - loss: 0.0681 - 28ms/epoch - 28ms/step
Epoch 40/100
1/1 - 0s - loss: 0.0701 - 26ms/epoch - 26ms/step
Epoch 41/100
1/1 - 0s - loss: 0.0716 - 24ms/epoch - 24ms/step
Epoch 42/100
1/1 - 0s - loss: 0.0725 - 24ms/epoch - 24ms/step
Epoch 43/100
1/1 - 0s - loss: 0.0726 - 27ms/epoch - 27ms/step
Epoch 44/100
1/1 - 0s - loss: 0.0720 - 34ms/epoch - 34ms/step
Epoch 45

Epoch 58/100
1/1 - 0s - loss: 0.0694 - 40ms/epoch - 40ms/step
Epoch 59/100
1/1 - 0s - loss: 0.0693 - 62ms/epoch - 62ms/step
Epoch 60/100
1/1 - 0s - loss: 0.0691 - 33ms/epoch - 33ms/step
Epoch 61/100
1/1 - 0s - loss: 0.0688 - 33ms/epoch - 33ms/step
Epoch 62/100
1/1 - 0s - loss: 0.0684 - 30ms/epoch - 30ms/step
Epoch 63/100
1/1 - 0s - loss: 0.0680 - 44ms/epoch - 44ms/step
Epoch 64/100
1/1 - 0s - loss: 0.0675 - 31ms/epoch - 31ms/step
Epoch 65/100
1/1 - 0s - loss: 0.0670 - 25ms/epoch - 25ms/step
Epoch 66/100
1/1 - 0s - loss: 0.0665 - 24ms/epoch - 24ms/step
Epoch 67/100
1/1 - 0s - loss: 0.0661 - 25ms/epoch - 25ms/step
Epoch 68/100
1/1 - 0s - loss: 0.0657 - 27ms/epoch - 27ms/step
Epoch 69/100
1/1 - 0s - loss: 0.0653 - 26ms/epoch - 26ms/step
Epoch 70/100
1/1 - 0s - loss: 0.0650 - 28ms/epoch - 28ms/step
Epoch 71/100
1/1 - 0s - loss: 0.0647 - 26ms/epoch - 26ms/step
Epoch 72/100
1/1 - 0s - loss: 0.0645 - 26ms/epoch - 26ms/step
Epoch 73/100
1/1 - 0s - loss: 0.0643 - 26ms/epoch - 26ms/step
Epoch 74

Epoch 87/100
1/1 - 0s - loss: 0.1227 - 20ms/epoch - 20ms/step
Epoch 88/100
1/1 - 0s - loss: 0.1226 - 30ms/epoch - 30ms/step
Epoch 89/100
1/1 - 0s - loss: 0.1225 - 19ms/epoch - 19ms/step
Epoch 90/100
1/1 - 0s - loss: 0.1224 - 32ms/epoch - 32ms/step
Epoch 91/100
1/1 - 0s - loss: 0.1223 - 26ms/epoch - 26ms/step
Epoch 92/100
1/1 - 0s - loss: 0.1222 - 22ms/epoch - 22ms/step
Epoch 93/100
1/1 - 0s - loss: 0.1221 - 31ms/epoch - 31ms/step
Epoch 94/100
1/1 - 0s - loss: 0.1221 - 26ms/epoch - 26ms/step
Epoch 95/100
1/1 - 0s - loss: 0.1220 - 27ms/epoch - 27ms/step
Epoch 96/100
1/1 - 0s - loss: 0.1219 - 25ms/epoch - 25ms/step
Epoch 97/100
1/1 - 0s - loss: 0.1218 - 25ms/epoch - 25ms/step
Epoch 98/100
1/1 - 0s - loss: 0.1217 - 19ms/epoch - 19ms/step
Epoch 99/100
1/1 - 0s - loss: 0.1216 - 27ms/epoch - 27ms/step
Epoch 100/100
1/1 - 0s - loss: 0.1215 - 16ms/epoch - 16ms/step
1/1 [==============================] - 1s 579ms/step
------------------------------------------------------------------------------

Epoch 1/100
1/1 - 3s - loss: 0.3292 - 3s/epoch - 3s/step
Epoch 2/100
1/1 - 0s - loss: 0.3189 - 28ms/epoch - 28ms/step
Epoch 3/100
1/1 - 0s - loss: 0.3089 - 28ms/epoch - 28ms/step
Epoch 4/100
1/1 - 0s - loss: 0.2990 - 31ms/epoch - 31ms/step
Epoch 5/100
1/1 - 0s - loss: 0.2893 - 31ms/epoch - 31ms/step
Epoch 6/100
1/1 - 0s - loss: 0.2797 - 29ms/epoch - 29ms/step
Epoch 7/100
1/1 - 0s - loss: 0.2701 - 28ms/epoch - 28ms/step
Epoch 8/100
1/1 - 0s - loss: 0.2606 - 29ms/epoch - 29ms/step
Epoch 9/100
1/1 - 0s - loss: 0.2511 - 27ms/epoch - 27ms/step
Epoch 10/100
1/1 - 0s - loss: 0.2416 - 25ms/epoch - 25ms/step
Epoch 11/100
1/1 - 0s - loss: 0.2321 - 26ms/epoch - 26ms/step
Epoch 12/100
1/1 - 0s - loss: 0.2225 - 25ms/epoch - 25ms/step
Epoch 13/100
1/1 - 0s - loss: 0.2130 - 48ms/epoch - 48ms/step
Epoch 14/100
1/1 - 0s - loss: 0.2034 - 53ms/epoch - 53ms/step
Epoch 15/100
1/1 - 0s - loss: 0.1938 - 33ms/epoch - 33ms/step
Epoch 16/100
1/1 - 0s - loss: 0.1842 - 33ms/epoch - 33ms/step
Epoch 17/100
1/1 - 0s

...

Epoch 3/100
1/1 - 0s - loss: 0.5560 - 29ms/epoch - 29ms/step
Epoch 4/100
1/1 - 0s - loss: 0.5416 - 27ms/epoch - 27ms/step
Epoch 5/100
1/1 - 0s - loss: 0.5274 - 28ms/epoch - 28ms/step
Epoch 6/100
1/1 - 0s - loss: 0.5132 - 29ms/epoch - 29ms/step
Epoch 7/100
1/1 - 0s - loss: 0.4990 - 27ms/epoch - 27ms/step
Epoch 8/100
1/1 - 0s - loss: 0.4848 - 29ms/epoch - 29ms/step
Epoch 9/100
1/1 - 0s - loss: 0.4705 - 29ms/epoch - 29ms/step
Epoch 10/100
1/1 - 0s - loss: 0.4561 - 30ms/epoch - 30ms/step
Epoch 11/100
1/1 - 0s - loss: 0.4417 - 28ms/epoch - 28ms/step
Epoch 12/100
1/1 - 0s - loss: 0.4271 - 28ms/epoch - 28ms/step
Epoch 13/100
1/1 - 0s - loss: 0.4124 - 28ms/epoch - 28ms/step
Epoch 14/100
1/1 - 0s - loss: 0.3976 - 28ms/epoch - 28ms/step
Epoch 15/100
1/1 - 0s - loss: 0.3826 - 27ms/epoch - 27ms/step
Epoch 16/100
1/1 - 0s - loss: 0.3675 - 28ms/epoch - 28ms/step
Epoch 17/100
1/1 - 0s - loss: 0.3524 - 28ms/epoch - 28ms/step
Epoch 18/100
1/1 - 0s - loss: 0.3372 - 26ms/epoch - 26ms/step
Epoch 19/100
1/

Epoch 32/100
1/1 - 0s - loss: 0.0806 - 29ms/epoch - 29ms/step
Epoch 33/100
1/1 - 0s - loss: 0.0802 - 31ms/epoch - 31ms/step
Epoch 34/100
1/1 - 0s - loss: 0.0798 - 30ms/epoch - 30ms/step
Epoch 35/100
1/1 - 0s - loss: 0.0794 - 29ms/epoch - 29ms/step
Epoch 36/100
1/1 - 0s - loss: 0.0790 - 28ms/epoch - 28ms/step
Epoch 37/100
1/1 - 0s - loss: 0.0786 - 30ms/epoch - 30ms/step
Epoch 38/100
1/1 - 0s - loss: 0.0782 - 30ms/epoch - 30ms/step
Epoch 39/100
1/1 - 0s - loss: 0.0779 - 29ms/epoch - 29ms/step
Epoch 40/100
1/1 - 0s - loss: 0.0776 - 28ms/epoch - 28ms/step
Epoch 41/100
1/1 - 0s - loss: 0.0773 - 38ms/epoch - 38ms/step
Epoch 42/100
1/1 - 0s - loss: 0.0771 - 30ms/epoch - 30ms/step
Epoch 43/100
1/1 - 0s - loss: 0.0769 - 27ms/epoch - 27ms/step
Epoch 44/100
1/1 - 0s - loss: 0.0766 - 30ms/epoch - 30ms/step
Epoch 45/100
1/1 - 0s - loss: 0.0764 - 29ms/epoch - 29ms/step
Epoch 46/100
1/1 - 0s - loss: 0.0762 - 30ms/epoch - 30ms/step
Epoch 47/100
1/1 - 0s - loss: 0.0760 - 29ms/epoch - 29ms/step
Epoch 48

Epoch 61/100
1/1 - 0s - loss: 0.0971 - 29ms/epoch - 29ms/step
Epoch 62/100
1/1 - 0s - loss: 0.0968 - 27ms/epoch - 27ms/step
Epoch 63/100
1/1 - 0s - loss: 0.0966 - 26ms/epoch - 26ms/step
Epoch 64/100
1/1 - 0s - loss: 0.0964 - 28ms/epoch - 28ms/step
Epoch 65/100
1/1 - 0s - loss: 0.0962 - 29ms/epoch - 29ms/step
Epoch 66/100
1/1 - 0s - loss: 0.0960 - 28ms/epoch - 28ms/step
Epoch 67/100
1/1 - 0s - loss: 0.0959 - 29ms/epoch - 29ms/step
Epoch 68/100
1/1 - 0s - loss: 0.0958 - 27ms/epoch - 27ms/step
Epoch 69/100
1/1 - 0s - loss: 0.0957 - 27ms/epoch - 27ms/step
Epoch 70/100
1/1 - 0s - loss: 0.0956 - 26ms/epoch - 26ms/step
Epoch 71/100
1/1 - 0s - loss: 0.0955 - 27ms/epoch - 27ms/step
Epoch 72/100
1/1 - 0s - loss: 0.0955 - 29ms/epoch - 29ms/step
Epoch 73/100
1/1 - 0s - loss: 0.0954 - 27ms/epoch - 27ms/step
Epoch 74/100
1/1 - 0s - loss: 0.0954 - 28ms/epoch - 28ms/step
Epoch 75/100
1/1 - 0s - loss: 0.0953 - 20ms/epoch - 20ms/step
Epoch 76/100
1/1 - 0s - loss: 0.0953 - 36ms/epoch - 36ms/step
Epoch 77

Epoch 90/100
1/1 - 0s - loss: 0.0536 - 28ms/epoch - 28ms/step
Epoch 91/100
1/1 - 0s - loss: 0.0535 - 28ms/epoch - 28ms/step
Epoch 92/100
1/1 - 0s - loss: 0.0535 - 28ms/epoch - 28ms/step
Epoch 93/100
1/1 - 0s - loss: 0.0534 - 28ms/epoch - 28ms/step
Epoch 94/100
1/1 - 0s - loss: 0.0533 - 32ms/epoch - 32ms/step
Epoch 95/100
1/1 - 0s - loss: 0.0533 - 28ms/epoch - 28ms/step
Epoch 96/100
1/1 - 0s - loss: 0.0532 - 28ms/epoch - 28ms/step
Epoch 97/100
1/1 - 0s - loss: 0.0531 - 29ms/epoch - 29ms/step
Epoch 98/100
1/1 - 0s - loss: 0.0531 - 28ms/epoch - 28ms/step
Epoch 99/100
1/1 - 0s - loss: 0.0530 - 27ms/epoch - 27ms/step
Epoch 100/100
1/1 - 0s - loss: 0.0530 - 28ms/epoch - 28ms/step
1/1 [==============================] - 1s 558ms/step
-----------------------------------------------------------------------------------
--------------------------PARAMTER 14----------------------------------------------
Epoch 1/100
1/1 - 3s - loss: 0.1094 - 3s/epoch - 3s/step
Epoch 2/100
1/1 - 0s - loss: 0.1067 - 3

Epoch 16/100
1/1 - 0s - loss: 0.1182 - 30ms/epoch - 30ms/step
Epoch 17/100
1/1 - 0s - loss: 0.1132 - 28ms/epoch - 28ms/step
Epoch 18/100
1/1 - 0s - loss: 0.1082 - 27ms/epoch - 27ms/step
Epoch 19/100
1/1 - 0s - loss: 0.1033 - 31ms/epoch - 31ms/step
Epoch 20/100
1/1 - 0s - loss: 0.0986 - 28ms/epoch - 28ms/step
Epoch 21/100
1/1 - 0s - loss: 0.0941 - 27ms/epoch - 27ms/step
Epoch 22/100
1/1 - 0s - loss: 0.0898 - 27ms/epoch - 27ms/step
Epoch 23/100
1/1 - 0s - loss: 0.0859 - 28ms/epoch - 28ms/step
Epoch 24/100
1/1 - 0s - loss: 0.0822 - 28ms/epoch - 28ms/step
Epoch 25/100
1/1 - 0s - loss: 0.0790 - 28ms/epoch - 28ms/step
Epoch 26/100
1/1 - 0s - loss: 0.0761 - 27ms/epoch - 27ms/step
Epoch 27/100
1/1 - 0s - loss: 0.0738 - 28ms/epoch - 28ms/step
Epoch 28/100
1/1 - 0s - loss: 0.0720 - 28ms/epoch - 28ms/step
Epoch 29/100
1/1 - 0s - loss: 0.0708 - 28ms/epoch - 28ms/step
Epoch 30/100
1/1 - 0s - loss: 0.0701 - 30ms/epoch - 30ms/step
Epoch 31/100
1/1 - 0s - loss: 0.0699 - 27ms/epoch - 27ms/step
Epoch 32

In [4]:
#plt.plot(data.index[train_size+time_steps:], data['Temperature'][train_size+time_steps:], label='Testing Data')
#plt.plot(data.index[train_size+time_steps:], test_predict[:,0], label='Testing Predictions')
#plt.legend()
#plt.show()